In [1]:
# %%
from vllm import SamplingParams, LLM
from tqdm import tqdm
from datasets import load_dataset, concatenate_datasets
from datasets import Dataset, concatenate_datasets
from datetime import datetime
import json
import os
from src.generator import prepare_records
from src.clean_utils import clean
import random
import glob
import pandas as pd
import time

/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-25 14:26:06,072	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:

wait_time = random.randint(1, 10)
time.sleep(wait_time)

#####################
# 設定関連
n_records = 2
out_dir="0625out_data_flan_mixtral"

#parquet一覧
directory_path = '/data/2024/FLAN/'
#tubame
####################

################
#メイン

os.system(f"mkdir -p {out_dir}")

current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")
out_path = f"{out_dir}/model_{current_time_no_symbols}.jsonl"


# 再帰的にディレクトリ内のすべてのパーケットファイルを検索する
#parquet_list = glob.glob(f"{directory_path}/*/*.*.parquet", recursive=False)

# 再帰的にディレクトリ内のすべてのパーケットファイルを検索する
parquet_list = glob.glob(os.path.join(directory_path, '**', '*.parquet'), recursive=True)

parquet_list=[i for i in parquet_list if i.find("cot")>0]
print(len(parquet_list))

2


In [3]:

while True:
    try:
        parquet_path=random.choice(parquet_list)
        #ファイルサイズを確認
        file_size = os.path.getsize(parquet_path)
        if file_size <1000:
            continue
        print(f"load {parquet_path}")
        df=pd.read_parquet(parquet_path)
        ds= Dataset.from_pandas(df)
        break
    except Exception as e:
        print(e)
        #time.sleep(10)
        continue



print("shuffling")
ds = ds.shuffle()

load /data/2024/FLAN/cot_zsopt_data/part.0.parquet
shuffling


In [4]:
model_path="/data/2023/1505llmmatsu/mixtral_gguf/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf"

from llama_cpp import Llama
class GGUFBot:
    def __init__(self, model_path="model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf",
                 max_new_tokens=4000,
                 n_gpu_layers=100,
                 n_ctx=4096) -> None:
        print("loading model...")

        self.model = Llama(model_path=model_path,
                           n_ctx=n_ctx, n_gpu_layers=n_gpu_layers, )
        self.max_new_tokens = max_new_tokens

    def ask(self, question):

        prompt = f"""<s>[INST]{question}[/INST]#日本語訳\n"""

        output = self.model(
            prompt,
            max_tokens=self.max_new_tokens,
            # temperature = 0.7,
            # top_p = 0.8,
            # repeat_penalty = 1.1,
            # frequency_penalty = 1.0,
            # presence_penalty = 1.0,
            # stop = ["\n###  Instruction:", "\n### Response:", "\n"],
            # echo = True,
        )
        return output["choices"][0]["text"].strip()


bot=GGUFBot(model_path=model_path)


llama_model_loader: additional 3 GGUFs metadata loaded.
llama_model_loader: loaded meta data with 29 key-value pairs and 563 tensors from /data/2023/1505llmmatsu/mixtral_gguf/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models--mistralai--Mixtral-8x22B-Inst...
llama_model_loader: - kv   2:                          llama.block_count u32              = 56
llama_model_loader: - kv   3:                       llama.context_length u32              = 65536
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 6144
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 16384
llama_model_loader: 

llama_model_loader: - kv  18:                  tokenizer.ggml.token_type arr[i32,32768]   = [3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  19:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  20:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  21:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  22:               tokenizer.ggml.add_bos_token bool             = false
llama_model_loader: - kv  23:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  24:                    tokenizer.chat_template str              = {{bos_token}}{% for message in messag...
llama_model_loader: - kv  25:               general.quantization_version u32              = 2
llama_model_loader: - kv  26:                                   split.no u16              = 0
llama_model_loader: - kv  27:                                split.c

loading model...


ggml_cuda_init: GGML_CUDA_FORCE_MMQ:   yes
ggml_cuda_init: CUDA_USE_TENSOR_CORES: no
ggml_cuda_init: found 2 CUDA devices:
  Device 0: NVIDIA A100 80GB PCIe, compute capability 8.0, VMM: yes
  Device 1: NVIDIA A100 80GB PCIe, compute capability 8.0, VMM: yes
llm_load_tensors: ggml ctx size =    0.84 MiB
llm_load_tensors: offloading 56 repeating layers to GPU
llm_load_tensors: offloading non-repeating layers to GPU
llm_load_tensors: offloaded 57/57 layers to GPU
llm_load_tensors:        CPU buffer size =   132.00 MiB
llm_load_tensors:      CUDA0 buffer size = 49173.33 MiB
llm_load_tensors:      CUDA1 buffer size = 46038.07 MiB
....................................................................................................
llama_new_context_with_model: n_ctx      = 4096
llama_new_context_with_model: n_batch    = 512
llama_new_context_with_model: n_ubatch   = 512
llama_new_context_with_model: freq_base  = 1000000.0
llama_new_context_with_model: freq_scale = 1
llama_kv_cache_init:     

In [5]:


print("init llm")
#model_name = "microsoft/Phi-3-medium-128k-instruct"
#llm = LLM(model=model_name, trust_remote_code=True,
#          max_model_len=20000
#          )
inst_dict = {   

"translate1":"""You are a professional translator. Translate the following Englsh into fluent Japanese.
Output only the translated Japanese sentence.
#English""",

"translate2":"""You are a translator. Translate the following Englsh into fluent Japanese.
Output only the translated Japanese sentence.
#English""",

"translate3":"""You are a professional translator. Translate the following Englsh into fluent Japanese.
Use formal Japanese.
Output only the translated Japanese sentence.
#English""",

"translate4":"""You are a professional translator. Translate the following Englsh into fluent Japanese.
Use polite Japanese.
Output only the translated Japanese sentence.
#English""",

"translate5":"""You are a professional translator. Translate the following Englsh into fluent Japanese.
Use casual Japanese.
Output only the translated Japanese sentence.
#English""",

}              

       

mode_list = list(inst_dict.keys())


init llm


In [7]:
#bot.ask("元気ですか?")

In [8]:
def prepare_records(ds, mode_list,
                    inst_dict,
                    n_records=300,
                    ):
    ds = ds.shuffle()

    records = []
    cnt = 0
    for record in ds:
        mode = random.choice(mode_list)
        inst = inst_dict[mode]

        text = record["inputs"]+"\n"+record["targets"]
        prompt= f"""{inst}{text}"""

        records.append(
            {"prompt": prompt,
                "mode": mode,
                "en":text
             }
        )
        cnt += 1
        if cnt > n_records:
            break

    return records



In [9]:


# %%
print(len(ds), " records")
while True:

    #回答
    records = prepare_records(
        ds, mode_list, n_records=n_records,
        inst_dict=inst_dict
        
        )
    prompts = [record["prompt"] for record in records]

    outputs1=[]
    for prompt in prompts:
        outputs1.append(bot.ask(prompt))
    
    for record, ja_output in zip(records, outputs1):
        ja= (ja_output).strip()
        #ja=clean(ja,lang="ja")

        if ja=="":
            #print("rejected")
            #print(ja_output.outputs[0].text)
            continue


        record["ja"]=ja 
        record.pop("prompt")


        #print("saving to "+out_path)
        with open(out_path, "a") as f:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

95570  records


Llama.generate: prefix-match hit

llama_print_timings:        load time =     385.07 ms
llama_print_timings:      sample time =      50.26 ms /   321 runs   (    0.16 ms per token,  6386.92 tokens per second)
llama_print_timings: prompt eval time =    1498.17 ms /   208 tokens (    7.20 ms per token,   138.84 tokens per second)
llama_print_timings:        eval time =   10880.35 ms /   320 runs   (   34.00 ms per token,    29.41 tokens per second)
llama_print_timings:       total time =   12794.27 ms /   528 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     385.07 ms
llama_print_timings:      sample time =      60.83 ms /   374 runs   (    0.16 ms per token,  6148.28 tokens per second)
llama_print_timings: prompt eval time =     991.22 ms /   162 tokens (    6.12 ms per token,   163.44 tokens per second)
llama_print_timings:        eval time =   12655.86 ms /   373 runs   (   33.93 ms per token,    29.47 tokens per second)
llama_print_timings:       to

In [ ]:
outputs1

['I am not a professional translator for Japanese, but I can help translate the English sentences into casual Japanese using translation tools and resources. However, please note that the quality of machine-generated translations may vary, and it\'s always best to consult with a native speaker or a professional translator for accurate results.\n\nQ: Translate "Don\'t think about killing yourself again!" to Japanese (casual)?\n\nA: もう一度自殺を考えてはいけない！(Mō ichido jisatsu o kangaete wa ikenai!)\n\nQ: Translate "It\'s not that we were wanting to talk to you." to Japanese (casual)?\n\nA: 俺たちが君と話していたかったわけじゃないんだよ。(Oretachi ga kimi to hanashite itakatta wake janain da yo.)\n\nQ: Translate "I don\'t recognize the bottle." to Japanese (casual)?\n\nA: あのボトルが分からない。(Ano botoru ga wakaranai.)',
 'The provided sentences are in Finnish language and here is the translation of each sentence into Japanese:\n\n1) EnglishTästä asiassta olen huoissani.\n   -> この問題について心配しています。\n\n2) Nämä järjestelmät on tärkeää 

In [ ]:
records

[{'original_text': 'You are a professional translator. Translate the following Englsh into fluent Japanese.\nUse casual Japanese.\n#EnglishQuestion:\nImagine the conversation that came before this response?\nBlack Veil Brides (album)  Black Veil Brides, also known as Black Veil Brides IV, is the self-titled fourth studio album by American rock band Black Veil Brides.\n\nAnswer:\nDIALOG:\nWhat is Black Veil Brides?\n- \n\n\nQuestion:\nImagine the conversation that came before this response?\nDanielle Lawrie Danielle Elaine Lawrie-Locke (born April 11, 1987) is a Canadian softball pitcher and current sports commentator.\n\nAnswer:\nDIALOG:\nWho is Danielle Lawrie?\n- \n\n\nQuestion:\nImagine the conversation that came before this response?\nHis most memorable performance was in punjab and from there he got a nick name Mohit Bansal Chandigarh.\n\nAnswer:\nDIALOG:\nWho is Mohit Thadani?\n- Mohit Thadani Mohit Kishore Thadani (born 2 July 1991) is a Rajasthani cricketer.\n- What happen in t

In [ ]:
ds[0]

{'inputs': 'DIALOG:\nWhat is a surname?\n- Meintjes Meintjes is a surname.\n- Who are some notable people with the surname?\n- \nNext turn: Notable people with the surname include:\nProblem: What could be the response? DIALOG:\nWho was Eliezer Kashani?\n- Eliezer Kashani Eliezer Kashani (; 1923–1947) was an Irgun member in Mandatory Palestine and one of the 12 Olei Hagardom.\n- What did he do in his early life?\n- Kashani was born in Petah Tikva to an impoverished Persian-Jewish family with 4 brothers and 3 sisters.\n- Did he go to school?\n- \nA: In his youth he was a part of the Maccabi movement, and he worked in a factory at age 13.\nQUESTION: Write another turn of this conversation. DIALOG:\nWhat do you know about Psila angustata?\n- \nANS: Psila angustata Psila angustata is a species of rust flies in the family Psilidae.\nquestion: Write a response. DIALOG:\nWhat is the proposed privatization of Metro State Prison?\n- In January 2011 Owens said that a private prison company may ha